In [23]:
#import of libraries
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import seaborn as sn
import matplotlib.pyplot as plt

In [24]:
df = pd.read_csv('data_audio_f_mean.csv')

In [25]:
df_test = df.iloc[:, 5:df.shape[1]]
df_test["target"].replace({"abnormal": "1", "normal": "0"}, inplace=True)
df_test = df_test.astype({"target": int})

In [26]:
df_test.head()

,target,spectogram,mel_spectogram,chromagram,chromagram_cq,chromagram_cens,mfcc,rms,spectral_centroid,spectral_bandwidth,spectral_contrast,spectral_flatness,roll_off_frequency,tonnetz,zero_crossing_rate
0,1,0.095985,0.032898,0.598389,0.335931,0.251026,-4.151321,0.007211,1391.829492,1770.435764,23.832949,0.000225,5660.559507,0.065789,0.053984
1,1,0.083133,0.034875,0.549378,0.386137,0.243251,-4.259758,0.007428,1127.738463,1574.488895,24.478842,0.000103,5120.156046,0.024099,0.043520
2,1,0.120339,0.027383,0.776584,0.321334,0.255388,-3.331495,0.006812,1804.463937,1813.068302,22.781830,0.000449,5696.131759,0.076063,0.084253
3,1,0.095124,0.046161,0.757323,0.373222,0.246073,-3.485691,0.008451,893.927245,1230.738677,21.108826,0.000061,3657.546948,0.044300,0.029228
4,1,0.132093,0.023357,0.862880,0.295585,0.250950,-3.539014,0.006496,2297.372342,1980.079305,22.923389,0.000795,6358.914758,0.077573,0.116914


In [12]:
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler

X = df_test.drop('target', axis=1)
y = df_test.target

print(f"'X' shape: {X.shape}")
print(f"'y' shape: {y.shape}")

pipeline = Pipeline([
    ('min_max_scaler', MinMaxScaler()),
    ('std_scaler', StandardScaler())
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

'X' shape: (54057, 13)
'y' shape: (54057,)


In [13]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

def print_score(clf, X_train, y_train, X_test, y_test, train=True):
    if train:
        pred = clf.predict(X_train)
        clf_report = pd.DataFrame(classification_report(y_train, pred, output_dict=True))
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_train, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_train, pred)}\n")
        
    elif train==False:
        pred = clf.predict(X_test)
        clf_report = pd.DataFrame(classification_report(y_test, pred, output_dict=True))
        print("Test Result:\n================================================")        
        print(f"Accuracy Score: {accuracy_score(y_test, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_test, pred)}\n")

In [14]:
from sklearn.svm import LinearSVC

model = LinearSVC(loss='hinge', dual=True)
model.fit(X_train, y_train)

print_score(model, X_train, y_train, X_test, y_test, train=True)
print_score(model, X_train, y_train, X_test, y_test, train=False)



Train Result:
Accuracy Score: 81.69%
_______________________________________________
CLASSIFICATION REPORT:
                      0       1  accuracy     macro avg  weighted avg
precision      0.816882     0.0  0.816882      0.408441      0.667296
recall         1.000000     0.0  0.816882      0.500000      0.816882
f1-score       0.899213     0.0  0.816882      0.449607      0.734551
support    30910.000000  6929.0  0.816882  37839.000000  37839.000000
_______________________________________________
Confusion Matrix: 
 [[30910     0]
 [ 6929     0]]

Test Result:
Accuracy Score: 81.68%
_______________________________________________
CLASSIFICATION REPORT:
                      0       1  accuracy     macro avg  weighted avg
precision      0.816808     0.0  0.816808      0.408404      0.667176
recall         1.000000     0.0  0.816808      0.500000      0.816808
f1-score       0.899169     0.0  0.816808      0.449584      0.734448
support    13247.000000  2971.0  0.816808  16218.000000

/home/ms/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/ms/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ms/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
y_pred = model.predict(X_test)

In [20]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.82      1.00      0.90     13247
           1       0.00      0.00      0.00      2971

    accuracy                           0.82     16218
   macro avg       0.41      0.50      0.45     16218
weighted avg       0.67      0.82      0.73     16218

